In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.sparse import csr_matrix
import random as rnd
import pickle
import sys


from datetime import datetime


from sklearn import preprocessing
from sklearn.model_selection import train_test_split




In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [3]:
def generate_random_array(median,std,size):
    output=[0]*size    
    for index in range(0,size//2):
        random_nr=rnd.uniform(-std,std)
        output[2*index]=median+random_nr
        output[2*index+1]=median-random_nr
    if(size % 2 != 0):
        output[size-1]=rnd.uniform(median-std,median+std)
    return output



In [4]:

traffic = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TrafficLabelling/combined.csv")
print(f'Number of Rows: {traffic.shape[0]}')
print(f'Number of Columns: {traffic.shape[1]}')
pd.set_option('display.max_colwidth', None)
traffic.head()

Number of Rows: 2660377
Number of Columns: 85


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,7/7/2017 3:30,109,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,7/7/2017 3:30,52,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,7/7/2017 3:30,34,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [5]:
# Remove duplicate entries
traffic.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

# reformat Timestamp to Unix time
# traffic[' Timestamp'] = pd.to_datetime(traffic[' Timestamp'])
# traffic[' Timestamp']= traffic[' Timestamp'].values.astype(np.int64) // 10 ** 9

# remove Timestamp column
traffic = traffic.drop(columns=[' Timestamp'])

# Remove constant columns
traffic = traffic.loc[:, traffic.apply(pd.Series.nunique) != 1]

# Remove column with NaN or Inf
traffic = traffic[~traffic.isin([np.nan, np.inf, -np.inf]).any(1)]

# remove Flow ID
traffic = traffic.drop(columns=['Flow ID'])

# reformat Label
traffic[' Label'] = traffic[' Label'].astype('category')
traffic[' Label'] = traffic[' Label'].cat.codes

# add new "is Attacker?" column
traffic[' is Attacker?'] = np.where(traffic[' Label'] == 0, 0, 1)



In [6]:
columns = list(traffic.columns)
columns


[' Source IP',
 ' Source Port',
 ' Destination IP',
 ' Destination Port',
 ' Protocol',
 ' Flow Duration',
 ' Total Fwd Packets',
 ' Total Backward Packets',
 'Total Length of Fwd Packets',
 ' Total Length of Bwd Packets',
 ' Fwd Packet Length Max',
 ' Fwd Packet Length Min',
 ' Fwd Packet Length Mean',
 ' Fwd Packet Length Std',
 'Bwd Packet Length Max',
 ' Bwd Packet Length Min',
 ' Bwd Packet Length Mean',
 ' Bwd Packet Length Std',
 'Flow Bytes/s',
 ' Flow Packets/s',
 ' Flow IAT Mean',
 ' Flow IAT Std',
 ' Flow IAT Max',
 ' Flow IAT Min',
 'Fwd IAT Total',
 ' Fwd IAT Mean',
 ' Fwd IAT Std',
 ' Fwd IAT Max',
 ' Fwd IAT Min',
 'Bwd IAT Total',
 ' Bwd IAT Mean',
 ' Bwd IAT Std',
 ' Bwd IAT Max',
 ' Bwd IAT Min',
 'Fwd PSH Flags',
 ' Fwd URG Flags',
 ' Fwd Header Length',
 ' Bwd Header Length',
 'Fwd Packets/s',
 ' Bwd Packets/s',
 ' Min Packet Length',
 ' Max Packet Length',
 ' Packet Length Mean',
 ' Packet Length Std',
 ' Packet Length Variance',
 'FIN Flag Count',
 ' SYN Flag Coun

In [7]:
# create ip address dict {ip_addr: node_id}
ips = []        # list of ip addresses.
for ip in traffic[' Source IP']:
    if ip not in ips:
        ips.append(ip)
        
for ip in traffic[' Destination IP']:
    if ip not in ips:
        ips.append(ip)
                
ips_dict = {ips[i]: i for i in range(len(ips))}   # dictionary of ip addresses { ip_addr: node_id }
print("Number of nodes: " + str(len(ips_dict)))


Number of nodes: 18598


In [8]:
# adding a group_size column to the original dataframe
traffic['group_size'] = traffic.groupby([' Source IP'])[' Destination IP'].transform(len)
print(traffic)

              Source IP   Source Port  ...  is Attacker?  group_size
0        104.16.207.165           443  ...             0          29
1         104.16.28.216            80  ...             0         114
2         104.16.28.216            80  ...             0         114
3         104.17.241.25           443  ...             0           2
4        104.19.196.102           443  ...             0         120
...                 ...           ...  ...           ...         ...
2660372   192.168.10.14         51114  ...             0      129475
2660373   192.168.10.16         24054  ...             0      114034
2660374  23.208.163.130           443  ...             0          51
2660375   192.168.10.14         51694  ...             0      129475
2660376   192.168.10.14         57949  ...             0      129475

[2657447 rows x 77 columns]


In [10]:
# define aggregation functions for all columns
aggregations = {col_name: "mean" for col_name in columns}
aggregations.pop(' Label', None)
aggregations.pop(' Source IP', None)
aggregations.pop(' Source Port', None)
aggregations.pop(' Destination IP', None)
aggregations.pop(' Destination Port', None)
aggregations.pop(' Protocol', None)
aggregations.pop(' is Attacker?', None)

aggregations_all = aggregations.copy()
aggregations_all.update({' Source Port': lambda x: len(set(x))})
aggregations_all.update({' Destination IP': lambda x: len(set(x))})
aggregations_all.update({' Destination Port': lambda x: len(set(x))})
aggregations_all.update({' Protocol': lambda x: len(set(x))})

aggregations.update({' is Attacker?': lambda x: 1 if 1 in set(x) else 0})
aggregations_all.update({' is Attacker?': lambda x: 1 if 1 in set(x) else 0})




print(aggregations)
print(aggregations_all)



{' Flow Duration': 'mean', ' Total Fwd Packets': 'mean', ' Total Backward Packets': 'mean', 'Total Length of Fwd Packets': 'mean', ' Total Length of Bwd Packets': 'mean', ' Fwd Packet Length Max': 'mean', ' Fwd Packet Length Min': 'mean', ' Fwd Packet Length Mean': 'mean', ' Fwd Packet Length Std': 'mean', 'Bwd Packet Length Max': 'mean', ' Bwd Packet Length Min': 'mean', ' Bwd Packet Length Mean': 'mean', ' Bwd Packet Length Std': 'mean', 'Flow Bytes/s': 'mean', ' Flow Packets/s': 'mean', ' Flow IAT Mean': 'mean', ' Flow IAT Std': 'mean', ' Flow IAT Max': 'mean', ' Flow IAT Min': 'mean', 'Fwd IAT Total': 'mean', ' Fwd IAT Mean': 'mean', ' Fwd IAT Std': 'mean', ' Fwd IAT Max': 'mean', ' Fwd IAT Min': 'mean', 'Bwd IAT Total': 'mean', ' Bwd IAT Mean': 'mean', ' Bwd IAT Std': 'mean', ' Bwd IAT Max': 'mean', ' Bwd IAT Min': 'mean', 'Fwd PSH Flags': 'mean', ' Fwd URG Flags': 'mean', ' Fwd Header Length': 'mean', ' Bwd Header Length': 'mean', 'Fwd Packets/s': 'mean', ' Bwd Packets/s': 'mean'

In [11]:
# low activity grouping
low_activities = traffic[traffic['group_size'] < 10]
low_activities = low_activities.groupby([' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Protocol'], as_index=False)
low_activities = low_activities.agg(aggregations)
# low_activities
low_activities = low_activities.groupby(' Source IP', as_index=False).agg(aggregations_all)
low_activities


,Source IP,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Source Port,Destination IP,Destination Port,Protocol,is Attacker?
0,0.0.0.0,3.668571e+07,2.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.054517,3.670000e+07,0.000000,3.670000e+07,3.670000e+07,3.670000e+07,3.670000e+07,0.000000,3.670000e+07,3.670000e+07,0.00,0.000,0.000000,0.000,0.000,0.000,0,0.000000,0.000000,0.054517,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.000,0,0.0,0.0,0.00,0,0,0.000,0.000000,0.0,0.0,0.000000,2.000000,0.0,0.000000,0.0,-1.00,-1.00,0.000000,0.0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,1,1,1,1,0
1,1.1.70.73,4.511675e+04,2.000000,0.250000,3.0,0.0,1.5,1.5,1.5,0.000000,0.0,0.0,0.0,0.0,8.720930e+03,4244.011538,2.269288e+04,31693.763375,4.510375e+04,2.820000e+02,4.511675e+04,4.511675e+04,0.000000,4.511675e+04,4.511675e+04,0.00,0.000,0.000000,0.000,0.000,0.000,0,58.000000,8.000000,4242.617935,1.393604,1.5,1.5,1.5,0.000000,0.00,0.0,0.000,0,0.0,1.0,0.25,0,0,0.000,2.250000,1.5,0.0,58.000000,2.000000,3.0,0.250000,0.0,4454.25,59.25,0.250000,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,4,1,0
2,1.193.219.21,6.480000e+02,3.000000,0.000000,258.0,0.0,258.0,0.0,86.0,148.956369,0.0,0.0,0.0,0.0,3.981481e+05,4629.629630,3.240000e+02,452.548340,6.440000e+02,4.000000e+00,6.480000e+02,3.240000e+02,452.548340,6.440000e+02,4.000000e+00,0.00,0.000,0.000000,0.000,0.000,0.000,0,96.000000,0.000000,4629.629630,0.000000,0.0,258.0,64.5,129.000000,16641.00,0.0,0.000,0,0.0,1.0,0.00,0,0,0.000,86.000000,86.0,0.0,96.000000,3.000000,258.0,0.000000,0.0,61.00,-1.00,1.000000,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,0
3,1.193.219.24,1.038322e+05,2.000000,0.500000,258.0,0.0,258.0,0.0,129.0,182.433549,0.0,0.0,0.0,0.0,5.376194e+07,416805.849036,2.604478e+04,51963.090590,1.038094e+05,1.187500e+01,1.038251e+05,1.038251e+05,0.000000,1.038251e+05,1.038251e+05,103484.25,51742.125,73173.884587,103483.875,0.375,0.875,0,64.000000,10.000000,416759.253610,46.595426,0.0,258.0,150.5,143.502831,20801.25,0.0,0.875,0,0.0,1.0,0.00,0,0,0.125,221.450000,129.0,0.0,64.000000,2.000000,258.0,0.500000,0.0,65.00,-0.75,0.125000,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,4,1,0
4,1.226.51.14,4.217540e+04,1.400000,0.600000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,15641.023359,4.217540e+04,0.000000,4.217540e+04,4.217540e+04,4.212860e+04,4.212860e+04,0.000000,4.212860e+04,4.212860e+04,0.00,0.000,0.000000,0.000,0.000,0.000,0,44.800000,19.200000,7824.395543,7816.627816,0.0,0.0,0.0,0.000000,0.00,0.0,0.000,0,0.0,1.0,0.60,0,0,0.600,0.000000,0.0,0.0,44.800000,1.400000,0.0,0.600000,0.0,155.20,866.00,0.000000,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [12]:
# attackers grouping
atk_activities = traffic[traffic[' is Attacker?'] == 1]
atk_activities = atk_activities.groupby([' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Protocol'], as_index=False)
atk_activities = atk_activities.agg(aggregations)

atk_activities = atk_activities.groupby(' Source IP', as_index=False).agg(aggregations_all)

atk_activities

,Source IP,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Source Port,Destination IP,Destination Port,Protocol,is Attacker?
0,172.16.0.1,5.268084e+06,2.010928,1.924461,38.841843,1758.341559,21.956343,1.251133,5.671120,7.858779,1037.397066,4.488321,331.157161,460.847793,217640.487688,58521.132837,5.641924e+05,1.550249e+06,4.969173e+06,28571.565225,5.139942e+06,9.816261e+05,1.982674e+06,4.904667e+06,115287.601370,1.368201e+06,3.095749e+05,4.860591e+05,1.222137e+06,90605.722280,0.009758,0,57.616754,47.721899,35056.583354,23464.549483,1.114579,1038.338797,169.152003,353.477936,917727.565908,0.013355,0.009758,0,0.856574,0.130250,0.007001,0,0,0.906924,189.075041,5.671120,331.157161,57.616754,2.010928,38.841843,1.924461,1758.341559,12793.989509,144.122876,0.742171,29.790743,25389.152942,2270.218706,26988.551947,23742.665367,4.446885e+06,473099.371105,4.791899e+06,4.110887e+06,48407,2,1000,3,1
1,192.168.10.12,3.013027e+07,5.000000,5.000000,161.000000,128.000000,161.000000,0.000000,17.888889,53.666667,128.000000,0.000000,14.222222,42.666667,4.800454,17.423337,1.799611e+06,2.450991e+06,5.128946e+06,28969.000000,2.560000e+07,3.198221e+06,2.634245e+06,5.100000e+06,117.000000,3.010000e+07,3.759477e+06,2.322569e+06,5.150000e+06,318.500000,0.000000,0,164.000000,164.000000,8.711669,8.711669,0.000000,161.000000,15.210526,45.891876,4212.128655,0.000000,0.000000,0,0.500000,0.500000,0.500000,0,0,1.000000,16.055556,17.888889,14.222222,164.000000,5.000000,161.000000,5.000000,128.000000,14718.500000,110.000000,0.500000,32.000000,31839.100000,11126.267980,51587.500000,25455.500000,5.100000e+06,17470.636005,5.100000e+06,5.050000e+06,1,1,1,1,1
2,192.168.10.14,4.012817e+05,4.755981,5.057416,4431.124402,107.736842,700.636364,0.000000,204.045950,198.270746,87.186603,2.009569,29.362901,44.737745,47960.846025,93.667237,7.676260e+04,1.150069e+05,2.302638e+05,177.090909,4.011073e+05,1.874357e+05,3.316208e+04,2.314064e+05,168935.693780,4.001571e+05,1.950188e+05,3.872412e+04,2.304383e+05,169237.483254,0.000000,0,113.818182,109.129187,46.028187,47.639050,0.000000,702.645933,89.350356,165.367536,271961.439215,0.000000,0.000000,0,1.000000,0.000000,0.000000,0,0,0.382775,94.335594,204.045950,29.362901,113.818182,4.755981,4431.124402,5.057416,107.736842,8192.000000,207.535885,3.086124,22.679426,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,209,1,1,1,1
3,192.168.10.15,4.577676e+05,4.272222,4.361111,3651.752778,96.222222,530.394444,0.050000,156.466963,172.651338,79.565278,2.383333,27.916924,41.596068,107467.457969,11879.528424,8.868100e+04,1.305477e+05,2.566674e+05,201.626389,4.575612e+05,2.173277e+05,3.216347e+04,2.579164e+05,199004.854167,4.565289e+05,2.249183e+05,3.771750e+04,2.568744e+05,199263.555556,0.000000,0,105.122222,94.455556,5940.144412,5939.384012,0.050000,532.727778,70.267443,136.552360,296331.934105,0.000000,0.000000,0,0.991667,0.008333,0.000

In [13]:
basic_flows = traffic.groupby([' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Protocol'], as_index=False)

basic_flows = basic_flows.agg(aggregations)
basic_flows

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0.0.0.0,0,224.0.0.22,0,0,36685713.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.054517,36700000.0,0.0000,36700000.0,36700000.0,36700000.0,36700000.0,0.0,36700000.0,36700000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054517,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.1.70.73,80,192.168.10.15,52772,6,344.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,34883.720930,5813.953488,344.0,0.0000,344.0,344.0,344.0,344.0,0.0,344.0,344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,5813.953488,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,2.0,12.0,0.0,0.0,4380.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.1.70.73,80,192.168.10.17,48301,6,315.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,6349.206349,315.0,0.0000,315.0,315.0,315.0,315.0,0.0,315.0,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,6349.206349,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.1.70.73,80,192.168.10.19,35264,6,179391.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,16.723247,89695.5,126775.0535,179339.0,52.0,179391.0,179391.0,0.0,179391.0,179391.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,32.0,11.148831,5.574416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,1.0,0.0,4677.0,240.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.1.70.73,80,192.168.10.19,35275,6,417.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4796.163070,417.0,0.0000,417.0,417.0,417.0,417.0,0.0,417.0,417.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,4796.163070,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382955,99.192.248.32,80,192.168.10.5,49372,6,55.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,218181.818182,36363.636364,55.0,0.0000,55.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,18181.818182,18181.818182,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,1.0,6.0,1.0,6.0,46.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1382956,99.192.248.32,80,192.168.10.5,49419,6,45.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,266666.666667,44444.444444,45.0,0.0000,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20

In [14]:
activity_flows = basic_flows.groupby([' Source IP'], as_index=False)
activity_flows = activity_flows.agg(aggregations_all)

# reformat Source IP
activity_flows = activity_flows.replace({" Source IP": ips_dict})
print(activity_flows)

atkers = activity_flows.groupby([' is Attacker?']).get_group(1)
non_atkers = activity_flows.groupby([' is Attacker?']).get_group(0)
print("Number of attackers: " + str(len(atkers)))
print("Number of non_atkers " + str(len(non_atkers)))

        Source IP   Flow Duration  ...   Protocol   is Attacker?
0           13147    3.668571e+07  ...          1              0
1             696    4.511675e+04  ...          1              0
2           15131    6.480000e+02  ...          1              0
3            1817    1.038322e+05  ...          1              0
4           13681    4.217540e+04  ...          1              0
...           ...             ...  ...        ...            ...
16508       13431    4.627100e+04  ...          1              0
16509        9872    4.732533e+04  ...          1              0
16510        2737    1.156000e+03  ...          1              0
16511       10465    8.496417e+03  ...          1              0
16512       10578    3.000000e+00  ...          1              0

[16513 rows x 75 columns]
Number of attackers: 10
Number of non_atkers 16503


In [27]:
# create low profiles dataframe
num_ips = len(ips_dict)
num_src_ips = len(activity_flows[' Source IP'])
num_noflow_ips = num_ips - num_src_ips


no_flow_prof_columns = columns[1:-3]
print(no_flow_prof_columns)
num_low_ips = round(0.95*num_noflow_ips)   #### adjust benign/attackers ratio here
low_activities_prof = np.zeros((num_low_ips, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = low_activities[attr].median()
  std = np.std(low_activities[attr])
  low_activities_prof[:, col] = generate_random_array(median, std, num_low_ips)


low_profile_df = pd.DataFrame(low_activities_prof, columns = no_flow_prof_columns)
low_profile_df[' is Attacker?'] = [0 for i in range(num_low_ips)]
low_profile_df[' Source IP'] = [num_src_ips + i for i in range(num_low_ips)]
low_profile_df


[' Source Port', ' Destination IP', ' Destination Port', ' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Fwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count',

,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,is Attacker?,Source IP
0,0.961786,1.803220,4.220391,1.009999,4.074024e+04,1.309612,1.037452,-119.588008,13.135629,-120.689947,4.467946,-11.163380,-26.085509,15.646306,-0.019848,2.589041,0.553185,6.834876e+05,-129054.867610,219624.311306,145996.930987,-86827.651072,-279636.526770,1.052947e+06,-2135.149073,280372.346876,1.479793e+05,169827.708439,-11661.335221,13497.083453,-25354.916261,-49321.754981,-66851.659230,-0.185804,0.0,38.656653,8.051611,10027.710449,-8413.585627,0.163268,-99.328577,37.526458,54.596211,90789.105109,-0.023165,0.243681,0.0,-0.010537,0.999053,0.798135,0.0,0.0,0.155309,-55.529078,76.892257,-2.905637,43.017430,1.597478,1251.390528,1.305313,0.611289,1739.835178,-4674.117583,0.212381,16.932393,16933.519053,-5932.903331,-11569.484632,-50715.529348,1.011499e+06,24014.975432,1.023255e+06,0,16513
1,1.038214,0.196780,1.779609,0.990001,-4.058624e+04,1.690388,0.462548,137.588008,-10.135629,132.689947,1.532054,23.163380,26.085509,-12.646306,2.686515,0.410959,-0.553185,-8.348755e+04,282901.021450,-219489.977972,-145996.930987,86975.651072,279727.955341,-1.052939e+06,2142.149073,-280372.346876,-1.479713e+05,-169822.458439,11661.335221,-13497.083453,25354.916261,49321.754981,66851.659230,0.185804,0.0,37.343347,27.948389,73450.550421,28107.670847,5.836732,111.328577,-25.526458,-54.596211,-90789.105109,0.023165,-0.243681,0.0,0.010537,1.000947,0.201865,0.0,0.0,0.844691,73.529078,-64.892257,5.905637,32.982570,1.402522,-1233.390528,0.194687,2.388711,-1489.835178,5103.117583,0.120952,23.067607,-16933.519053,5932.903331,11569.484632,50715.529348,-1.011499e+06,-24014.975432,-1.023255e+06,0,16514
2,1.098076,1.079142,2.492162,1.003004,-1.073621e+06,1.644572,0.445169,-1078.732682,1.282635,17.003613,23.260705,-13.528687,-33.586709,-2.667405,0.727361,-12.380141,-1.017637,-8.483535e+06,139551.732109,314819.419559,-575920.136971,899038.775986,55231.704514,5.651054e+05,451857.768694,-216686.541105,-1.077858e+06,35383.641860,115367.909543,47221.886990,8591.645331,-3003.715217,74940.658525,0.291744,0.0,26.942180,13.393903,-93383.663402,39940.698510,0.661278,43.177571,3.463432,66.976432,59706.094666,-0.040214,-0.209378,0.0,0.007258,0.964708,0.175928,0.0,0.0,0.375310,4.816922,4.850543,1.058868,31.752681,1.073503,-845.786510,0.950783,1.909189,841.666892,-8092.527418,1.030022,24.395754,24929.694860,9622.846928,59723.858379,-18606.694796,4.496308e+05,-78265.407234,6.899358e+05,0,16515
3,0.901924,0.920858,3.507838,0.996996,1.073775e+06,1.355428,1.054831,1096.732682,1.717365,-5.003613,-17.260705,25.528687,33.586709,5.667405,1.939306,15.380141,1.017637,9.083535e+06,14294.421731,-314685.086225,575920.136971,-898890.775986,-55140.275942,-5.650974e+05,-451850.768694,216686.541105,1.077866e+06,-35378.391860,-115367.909543,-47221.886990,-8591.645331,3003.715217,-74940.658525,-0.291744,0

In [28]:
# create attacker profiles dataframe

num_atk_ips = num_noflow_ips-num_low_ips

atker_prof = np.zeros((num_atk_ips, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = atk_activities[attr].median()
  std = np.std(atk_activities[attr])
  atker_prof[:, col] = generate_random_array(median, std, num_atk_ips)


atker_profile_df = pd.DataFrame(atker_prof, columns = no_flow_prof_columns)
atker_profile_df[' is Attacker?'] = [1 for i in range(num_atk_ips)]
atker_profile_df[' Source IP'] = [num_src_ips+num_low_ips + i for i in range(num_atk_ips)]
atker_profile_df


,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,is Attacker?,Source IP
0,6865.494505,0.972613,-304.334396,1.544764,7.926562e+06,2.639930,4.529804,-67.144135,423.991905,563.312651,-0.776486,100.129857,176.450222,157.051017,2.813749,67.890385,113.479167,-99515.380877,-16482.928944,740477.748017,8.052554e+05,2.704090e+06,-546.231375,1.025523e+07,1.028158e+06,515045.662625,-2.141740e+05,170368.260452,1.170701e+07,2.336013e+05,-8.925850e+05,-5.331480e+05,183523.297158,-0.000881,0.0,66.237277,41.587951,-8916.510906,10869.053359,0.240136,786.343177,14.121056,136.271266,343200.536833,-0.000212,-0.003120,0.0,1.098587,0.145481,-0.009541,0.0,0.0,1.546085,53.556944,126.310161,-68.613067,141.273839,6.321105,2719.560652,2.903564,-338.653671,10265.374551,-304.550304,-0.723091,24.910483,34382.591950,7541.177882,-3499.708269,18037.324252,2.043299e+06,136111.470138,2.397145e+06,1,18494
1,-6477.494505,1.027387,306.334396,0.455236,-5.586908e+06,6.887001,5.860945,3925.396912,-204.085380,10.483926,0.894360,28.844823,6.599697,10.470782,1.915953,-19.480817,-31.118726,198523.234202,21459.483377,-511760.538272,-4.303868e+05,-1.652840e+06,955.919216,-9.340740e+06,-5.958362e+05,-444231.752819,7.320817e+05,104807.069424,-9.882281e+06,3.008919e+05,1.061785e+06,1.558590e+06,67413.230552,0.000881,0.0,162.821128,181.999336,11469.584341,-8445.572361,-0.122263,117.228708,112.359073,99.459292,88947.266938,0.000212,0.003120,0.0,0.717043,0.025713,0.017810,0.0,0.0,-0.043356,81.148682,2.664519,117.022635,87.784566,3.205825,1138.692126,7.487186,558.560196,6118.625449,635.400814,3.959706,21.329119,-19364.258616,-7541.177882,18518.041602,-3018.990918,-1.758547e+06,-136111.470138,-2.011195e+06,1,18495
2,-8670.348019,1.525986,-248.943482,1.576945,-7.683527e+06,0.613549,7.041708,4513.429566,188.200402,132.970653,-0.292719,29.052698,17.222265,99.252425,1.828155,-46.745408,-16.736258,-70011.628224,34939.394802,41534.956807,-5.172800e+05,-1.125988e+05,-12066.223432,-2.906476e+05,6.510770e+05,-434865.965770,-1.830198e+06,87812.522638,9.795082e+06,-1.252749e+05,-6.672058e+05,-3.757637e+05,83863.242792,0.000279,0.0,198.601271,38.494050,-8681.997279,14740.788261,1.438776,694.213682,23.127128,26.196935,57581.275599,0.002704,-0.000756,0.0,0.615959,0.288003,0.333817,0.0,0.0,2.078642,101.916581,20.853164,5.744122,44.852800,3.261225,883.025248,6.365859,435.879806,7324.895215,239.416773,-2.020266,23.322817,29667.788272,6603.677063,9742.367593,-7475.726938,2.367598e+06,-1344.456163,-8.057711e+05,1,18496
3,9058.348019,0.474014,250.943482,0.423055,1.002318e+07,8.913381,3.349042,-655.176788,31.706124,440.825925,0.410592,99.921982,165.827655,68.269374,2.901547,95.154976,99.096699,169019.481549,-29962.840370,187182.252938,8.921485e+05,1.163849e+06,12475.911274,1.205136e+06,-2.187552e+05,505679.875576,2.348105e+06,187362.807238,-7.970353e+06,6.597680e+05,8.364058e+05,1.4012

In [29]:
# combines all the dataframes
activity_flows_all = activity_flows.append(low_profile_df, ignore_index=True)
activity_flows_all = activity_flows_all.append(atker_profile_df, ignore_index=True)

In [30]:
atkers = activity_flows_all.groupby([' is Attacker?']).get_group(1)
non_atkers = activity_flows_all.groupby([' is Attacker?']).get_group(0)
print("Number of attackers: " + str(len(atkers)))
print("Number of non_atkers " + str(len(non_atkers)))

Number of attackers: 114
Number of non_atkers 18484


In [31]:
activity_flows_all = activity_flows_all.sort_values(by=' Source IP', ignore_index=True)
activity_flows_all

,Source IP,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Source Port,Destination IP,Destination Port,Protocol,is Attacker?
0,0,2.958276e+02,1.655172,0.551724,9.241379,2.275862,6.344828,4.137931,4.873563,1.274152,2.275862,2.275862,2.275862,0.000000,7.443734e+05,173200.060082,254.298851,3.268862e+01,2.907586e+02,233.827586,2.620000e+02,232.482759,39.402916,2.603448e+02,204.620690,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,33.103448,11.034483,154810.571641,18389.488437,4.137931,6.344828,4.579310,0.986954,14.124138,0.000000,0.000000,0.0,0.000000,1.000000,0.482759,0.0,0.0,0.413793,6.620690,4.873563,2.275862,33.103448,1.655172,9.241379,0.551724,2.275862,31.586207,174.034483,0.482759,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,1.000000,8.00000,29.000000,1.000000,0
1,1,2.968569e+04,1.122807,0.877193,5.842105,4.789474,5.315789,5.315789,5.315789,0.000000,4.789474,4.789474,4.789474,0.000000,3.480764e+05,60856.512853,29685.692982,0.000000e+00,2.968569e+04,29685.692982,2.962772e+04,29627.719298,0.000000,2.962772e+04,29627.719298,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,22.456140,17.543860,35588.410486,25268.102366,5.315789,5.315789,5.315789,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.877193,0.0,0.0,0.877193,7.973684,5.315789,4.789474,22.456140,1.122807,5.842105,0.877193,4.789474,29.008772,2768.675439,0.087719,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,1.000000,10.00000,114.000000,1.000000,0
2,2,1.920000e+02,1.500000,0.500000,9.000000,3.000000,6.000000,6.000000,6.000000,0.000000,3.000000,3.000000,3.000000,0.000000,1.936134e+05,32268.907562,192.000000,0.000000e+00,1.920000e+02,192.000000,1.750000e+02,175.000000,0.000000,1.750000e+02,175.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,30.000000,10.000000,17563.025212,14705.882355,6.000000,6.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.500000,0.0,0.0,0.500000,9.000000,6.000000,3.000000,30.000000,1.500000,9.000000,0.500000,3.000000,31.000000,164.000000,0.500000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,1.000000,2.00000,2.000000,1.000000,0
3,3,1.322275e+03,2.050847,0.279661,26.500000,1.449153,19.483051,4.576271,9.133333,7.515555,1.423729,1.423729,1.423729,0.000000,1.011438e+06,204326.933754,579.338983,5.730325e+02,1.303220e+03,255.038136,1.307114e+03,778.599576,595.660755,1.296072e+03,380.025424,1.271186e-02,1.271186e-02,0.000000,1.271186e-02,0.012712,0.025424,0.0,41.016949,5.593220,201936.127350,2390.806404,4.576271,19.483051,8.742857,7.009958,605.918402,0.000000,0.025424,0.0,0.000000,1.000000,0.152542,0.0,0.0,0.152542,11.812853,9.133333,1.423729,41.016949,2.050847,26.500000,0.279661,1.449153,31.533898,1701.580508,0.800847,20.000000,0.000000,0.000000,0.000000,0.0000

In [32]:
# (1) SELECT | ALL features
X = activity_flows_all.iloc[:, 1:(activity_flows_all.shape[1]-1)]
Y = activity_flows_all.iloc[:, -1]

In [33]:
# create hypergraph
connection = traffic.groupby([' Source IP', ' Destination IP'], as_index=False)
connection = connection.agg({' Flow Duration' : 'count'})
src_ip_ids = [ips_dict[ip] for ip in connection[' Source IP']]
dst_ip_ids = [ips_dict[ip] for ip in connection[' Destination IP']]
edges = list(zip(src_ip_ids, dst_ip_ids))
hypergraph = {i: edges[i] for i in range(len(edges))}

In [34]:
# # generate features
features = X
features = np.nan_to_num(features)
features = preprocessing.normalize(features)
features = csr_matrix(features)

# generate features from feature selection set
# X = X.fillna(0)
# selected_features = selectFeatures(X, Y)
# print(f'Selected Features "from All": {selected_features}')
# features = X[selected_features]
# print("Shape of features: ", features.shape)

# features = preprocessing.normalize(features)
# features = csr_matrix(features)


In [35]:
# generate labels
labels = Y

In [39]:
# train/test split

splits = {}
attackers_ids = list(atkers.index)
V = len(ips)
node_sorted = list(range(V))
rnd.shuffle(node_sorted)
train_size = round(0.8*V)
test_size = V - train_size
train_set, test_set = train_test_split(node_sorted, train_size=train_size, test_size=test_size)



splits['train'] = train_set
splits['test'] = test_set
splits


{'test': [8642,
  14698,
  17142,
  5377,
  18401,
  6959,
  2117,
  17383,
  11929,
  7753,
  10653,
  5962,
  10727,
  1497,
  6450,
  8382,
  13065,
  18145,
  14160,
  14019,
  14683,
  4759,
  14522,
  3476,
  379,
  9981,
  5910,
  5121,
  18293,
  681,
  8185,
  15705,
  431,
  15529,
  15257,
  10668,
  11498,
  7514,
  3433,
  17705,
  10861,
  5519,
  16510,
  13909,
  13811,
  6027,
  9623,
  7448,
  12155,
  9005,
  258,
  4018,
  12452,
  10995,
  15782,
  1058,
  12070,
  15090,
  15896,
  8994,
  11226,
  1463,
  16758,
  10238,
  7032,
  14940,
  771,
  12357,
  8641,
  10577,
  12456,
  6405,
  6401,
  11765,
  6930,
  16793,
  7619,
  8285,
  13845,
  2974,
  5653,
  5948,
  10734,
  502,
  12567,
  13376,
  6345,
  6880,
  2351,
  2805,
  16880,
  15935,
  11034,
  5872,
  5464,
  10154,
  6570,
  396,
  3276,
  9241,
  216,
  2551,
  16586,
  1760,
  8627,
  16340,
  16056,
  4945,
  18421,
  14172,
  16674,
  18164,
  4528,
  1642,
  10181,
  1456,
  9992,
  11375,

In [37]:
train_atk_count = 0
for idx, train_id in enumerate(train_set):
    if labels[train_id] == 1:
      train_atk_count += 1

test_atk_count = 0
for idx, test_id in enumerate(test_set):
    if labels[test_id] == 1:
      test_atk_count += 1

print("Number of atker in train set: ", train_atk_count)
print("Number of atker in test set: ", test_atk_count)

Number of atker in train set:  93
Number of atker in test set:  21


In [38]:
# generate files
with open("hypergraph.pickle", "wb") as hypergraph_out:
    pickle.dump(hypergraph, hypergraph_out)
    hypergraph_out.close()
with open("features.pickle", "wb") as features_out:
    pickle.dump(features, features_out)
    features_out.close()
with open("labels.pickle", "wb") as labels_out:
    pickle.dump(labels, labels_out)
    labels_out.close()
with open("1.pickle", "wb") as splits_out:
    pickle.dump(splits, splits_out)
    splits_out.close()